# Fitbit to Jupyter Playbook

This notebook offers some tools to play around with your Fitbit data in Python.
Some hacks are used to get Fitbit API authentication running.


## Getting started: Step 1/6
If you do not already have a Fitbit API Application, create one at <https://dev.fitbit.com/apps/new>. You can fill all fields to your liking, but the "Callback URL" must be identical to the url of this notebook (e.g. https://hub.gke.mybinder.org/user/aartgoossens-strava-to-jupyter-bladibla/notebooks/fitbitplaybook.ipynb), otherwise the authentication flow will not work.

Documentation about the Fitbit API can be found [here](https://dev.fitbit.com/build/reference/web-api/).

## Getting started: Step 2/6
Now run the cell below and enter your Fitbit OAuth 2.0 Client ID and Client Secret at the prompts.

In [ ]:
import getpass
import os
from datetime import timedelta

import pandas as pd
from IPython.display import display, HTML, Markdown


%matplotlib inline


FITBIT_CLIENT_ID = input('Enter your Fitbit OAuth 2.0 Client: ')
FITBIT_CLIENT_SECRET = getpass.getpass('Enter your Fitbit Client Secret: ')

## Getting started: Step 3/6
Subsequently execute the following cell. This uses some HTML magic to fetch the url at which this notebook is running, to be able to redirect to it later.

In [ ]:
HTML('''
    <script type="text/javascript">
        IPython.notebook.kernel.execute("notebook_url = '" + window.location + "'")
    </script>''')

## Getting started: Step 4/6
Now execute the cell below and click the generated url. The variable `notebook_url` is generated by the magic HTML-code in the previous cell.

**Warning**: This will direct you away from this page and directly back afterwards but you will need to scroll down a bit to get back to the next step.

In [ ]:
url = (
    f'https://www.fitbit.com/oauth2/authorize?'
    f'client_id={FITBIT_CLIENT_ID}&'
    f'redirect_uri={notebook_url}&'
    f'approval_prompt=auto&'
    f'response_type=token&'
    f'scope=sleep+weight+profile+social+location+heartrate+settings+activity+nutrition&'
    f'token_type=Bearer&'
    f'expires_in=604800'
)

print('Open this url in a new tab or window:')
display(Markdown(f'<{url}>'))

## Getting started: Step 5/6
Subsequently execute the following cell. This (again) uses some HTML magic to fetch a code that Strava inserted into the callback (redirect) url.

In [ ]:
HTML('''
    <script type="text/javascript">
        IPython.notebook.kernel.execute("redirected_url = '" + window.location + "'")
    </script>''')

## Getting started: Step 6/6
And finally execute the following cell and paste the url you were being redirected to in the input field.

In [ ]:
qs = redirected_url.split('#')[1]
assert qs.startswith('access_token')
access_token_qs = qs.split('&')[0]
assert access_token_qs.startswith('access_token')
access_token = access_token_qs.split('=')[1]

...and you're done! You can take a look at (or edit) the example code or scroll all the way down and do your own thing.

## Example code

In [ ]:
import dateutil
import datetime

import requests

In [ ]:
now = datetime.datetime.now()
for days in range(-13, 1):
    dt = now + datetime.timedelta(days=days)

    date = dt.strftime("%Y-%m-%d")

    resp = requests.get(
        url=f'https://api.fitbit.com/1.2/user/-/sleep/date/{date}.json',
        headers={'Authorization': f'Bearer {access_token}'}
    )
    resp.raise_for_status()
    sleep_data = resp.json()
    
    try:
        start_datetime = dateutil.parser.parse(sleep_data['sleep'][0]['startTime'])
    except IndexError:
        print(f'No sleep recorded on {dt.date()}. Skipping...')
        continue
    start_date = start_datetime.strftime('%Y-%m-%d')
    start_time = start_datetime.strftime('%H:%M')

    end_datetime = dateutil.parser.parse(sleep_data['sleep'][0]['endTime'])
    end_date = end_datetime.strftime('%Y-%m-%d')
    end_time = end_datetime.strftime('%H:%M')


    resp = requests.get(
        url=f'https://api.fitbit.com/1/user/-/activities/heart/date/{start_date}/{end_date}/1sec/time/{start_time}/{end_time}.json',
        headers={'Authorization': f'Bearer {access_token}'}
    )
    hr_data = resp.json()

    datapoints = hr_data['activities-heart-intraday']['dataset']
    time = [i['time'] for i in datapoints]
    hr = [i['value'] for i in datapoints]
    df = pd.DataFrame(dict(hr=hr), index=pd.DatetimeIndex(time))

    hr_counts = df['hr'].value_counts().sort_index()

    for hr, count in hr_counts.iteritems():
        if count > 50: # This is quite arbitrary...
            break
    print(f'srHR on {dt.date()} (dow: {dt.weekday()}) was {hr}bpm')